In [1]:
import numpy as np
import matplotlib.pyplot as plt

# 参数设置
num_particles = 1000
domain_size = 10.0  # cm
disc_radius = 3.0  # cm
temperature = 1000  # K
mass_electron = 9.10938356e-31  # kg
mass_ion = 1.6726219e-27  # kg (质子质量)
charge_electron = -1.60217662e-19  # C
charge_ion = 1.60217662e-19  # C
k_B = 1.38064852e-23  # J/K
num_steps = 1000
dt = 8.3e-10  # s (8.3e-7 / 1000)
epsilon_0 = 8.854187817e-12  # F/m (真空介电常数)
debye_length = np.sqrt(epsilon_0 * k_B * temperature / (charge_electron**2))  # 德拜长度 (m)

# 转换德拜长度到网格单元
grid_size = 60
cell_size = domain_size / grid_size
debye_radius_grid = int(np.ceil(debye_length / cell_size))

# 初始化粒子位置和速度
def initialize_particles(num_particles, disc_radius, mass, temperature):
    positions = np.random.uniform(-disc_radius, disc_radius, (num_particles, 2))
    velocities = np.random.normal(0, np.sqrt(k_B * temperature / mass), (num_particles, 2))
    distances = np.linalg.norm(positions, axis=1)
    inside_disc = distances <= disc_radius
    return positions[inside_disc], velocities[inside_disc]

# 电子和离子初始化
electron_positions, electron_velocities = initialize_particles(num_particles, disc_radius, mass_electron, temperature)
ion_positions, ion_velocities = initialize_particles(num_particles, disc_radius, mass_ion, temperature)
num_particles_e = len(electron_positions)
num_particles_i = len(ion_positions)
charge_electron_array = np.full(num_particles_e, charge_electron)
charge_ion_array = np.full(num_particles_i, charge_ion)
mass_electron_array = np.full(num_particles_e, mass_electron)
mass_ion_array = np.full(num_particles_i, mass_ion)

# 电势和电场初始化
phi = np.zeros((grid_size, grid_size))
electric_field = np.zeros((grid_size, grid_size, 2))

# 泊松方程求解
def solve_poisson(phi, rho):
    for _ in range(100):  # 迭代次数
        phi_new = phi.copy()
        phi_new[1:-1, 1:-1] = 0.25 * (phi[:-2, 1:-1] + phi[2:, 1:-1] + phi[1:-1, :-2] + phi[1:-1, 2:] - rho[1:-1, 1:-1] * cell_size**2)
        phi = phi_new
    return phi

# 计算电场
def compute_electric_field(phi):
    ex = (phi[:-2, 1:-1] - phi[2:, 1:-1]) / (2 * cell_size)
    ey = (phi[1:-1, :-2] - phi[1:-1, 2:]) / (2 * cell_size)
    return ex, ey

# 计算库仑加速度（只考虑德拜半径内的粒子）
def compute_coulomb_acceleration(positions, charges, masses, debye_radius_grid):
    num_particles = len(positions)
    acceleration = np.zeros_like(positions)
    
    for i in range(num_particles):
        dx = positions - positions[i]
        distance = np.linalg.norm(dx, axis=1)
        within_debye_radius = distance <= debye_radius_grid * cell_size
        
        for j in np.where(within_debye_radius)[0]:
            if j == i:
                continue
            r = distance[j]
            force = charges[i] * charges[j] / (4 * np.pi * epsilon_0 * r**3)
            acceleration[i] += force * dx[j] / masses[i]
        
    return acceleration

# 计算电荷密度
def compute_charge_density(positions, charge, grid_size, domain_size, cell_size):
    rho = np.zeros((grid_size, grid_size))
    bin_edges = [np.linspace(-domain_size / 2, domain_size / 2, grid_size + 1)] * 2
    hist, _, _ = np.histogram2d(positions[:, 0], positions[:, 1], bins=bin_edges, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
    rho = hist * charge
    return rho

# 初始化电荷密度
rho_e = compute_charge_density(electron_positions, charge_electron, grid_size, domain_size, cell_size)
rho_i = compute_charge_density(ion_positions, charge_ion, grid_size, domain_size, cell_size)
rho = rho_e + rho_i

# 求解初始电势
phi = solve_poisson(phi, rho)

# 计算初始电场
ex, ey = compute_electric_field(phi)
electric_field[1:-1, 1:-1, 0] = ex
electric_field[1:-1, 1:-1, 1] = ey

# 模拟
for step in range(num_steps + 1):  # 包括初始状态
    if step > 0:
        # 计算电子和离子之间的库仑加速度
        electron_acceleration = compute_coulomb_acceleration(electron_positions, charge_electron_array, mass_electron_array, debye_radius_grid)
        ion_acceleration = compute_coulomb_acceleration(ion_positions, charge_ion_array, mass_ion_array, debye_radius_grid)

        # 更新电子位置和速度
        temp_positions_e = electron_positions.copy()
        electron_positions += electron_velocities * dt + 0.5 * electron_acceleration * dt**2
        electron_velocities += electron_acceleration * dt
        
        # 确保电子不会离开圆盘边界
        distances_e = np.linalg.norm(electron_positions, axis=1)
        outside_disc_e = distances_e > disc_radius
        electron_positions[outside_disc_e] = temp_positions_e[outside_disc_e]
        electron_velocities[outside_disc_e] *= -1  # 反弹

        # 更新离子位置和速度
        temp_positions_i = ion_positions.copy()
        ion_positions += ion_velocities * dt + 0.5 * ion_acceleration * dt**2
        ion_velocities += ion_acceleration * dt
        
        # 确保离子不会离开圆盘边界
        distances_i = np.linalg.norm(ion_positions, axis=1)
        outside_disc_i = distances_i > disc_radius
        ion_positions[outside_disc_i] = temp_positions_i[outside_disc_i]
        ion_velocities[outside_disc_i] *= -1  # 反弹

        # 计算新的电荷密度
        rho_e = compute_charge_density(electron_positions, charge_electron, grid_size, domain_size, cell_size)
        rho_i = compute_charge_density(ion_positions, charge_ion, grid_size, domain_size, cell_size)
        rho = rho_e + rho_i

        # 解决泊松方程
        phi = solve_poisson(phi, rho)

        # 计算电场
        ex, ey = compute_electric_field(phi)
        electric_field[1:-1, 1:-1, 0] = ex
        electric_field[1:-1, 1:-1, 1] = ey

    # 可视化
    if step % 1 == 0:
        fig, axs = plt.subplots(1, 3, figsize=(18, 6))
        axs[0].cla()
        axs[1].cla()
        axs[2].cla()
        
        hist_e, xedges, yedges = np.histogram2d(electron_positions[:, 0], electron_positions[:, 1], bins=grid_size, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
        hist_i, _, _ = np.histogram2d(ion_positions[:, 0], ion_positions[:, 1], bins=grid_size, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
        
        im1 = axs[0].imshow(hist_i.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='Reds', vmin=0, vmax=np.max(hist_i))
        axs[0].set_title('Ion Distribution')

        im2 = axs[1].imshow(hist_e.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='Blues', vmin=0, vmax=np.max(hist_e))
        axs[1].set_title('Electron Distribution')

        magnitude_e_field = np.sqrt(electric_field[:, :, 0]**2 + electric_field[:, :, 1]**2)
        im3 = axs[2].imshow(magnitude_e_field.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='plasma', vmin=0, vmax=np.max(magnitude_e_field))
        axs[2].set_title('Electric Field Magnitude')

        fig.colorbar(im1, ax=axs[0], orientation='vertical', label='Number of Ions')
        fig.colorbar(im2, ax=axs[1], orientation='vertical', label='Number of Electrons')
        fig.colorbar(im3, ax=axs[2], orientation='vertical', label='Electric Field Magnitude')

        plt.tight_layout()
        plt.savefig(f'step_{step:04d}.png')  # 保存图像
        plt.clf()  # 清理当前图像
        plt.close(fig)
print('所有图像已保存')


KeyboardInterrupt: 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange

# 参数设置
num_particles = 100000
domain_size = 10.0  # cm
disc_radius = 3.0  # cm
temperature = 1000  # K
mass_electron = 9.10938356e-31  # kg
mass_ion = 1.6726219e-27  # kg (质子质量)
charge_electron = -1.60217662e-19  # C
charge_ion = 1.60217662e-19  # C
k_B = 1.38064852e-23  # J/K
num_steps = 1000
dt = 8.3e-10  # s (8.3e-7 / 1000)
epsilon_0 = 8.854187817e-12  # F/m (真空介电常数)

# 计算粒子密度（m^-3）
volume = (2 * disc_radius / 100)**2  # 转换为m²
n = num_particles / volume  # 粒子密度 #只考虑初始情况等离子体密度

# 计算德拜长度
debye_length = np.sqrt(epsilon_0 * k_B * temperature / (n * charge_electron**2))

# 转换德拜长度到网格单元
grid_size = 60
cell_size = domain_size / grid_size  # cm
debye_radius_grid = int(np.ceil(debye_length / (cell_size / 100)))  # 转换为cm，再转换为网格单元


# 初始化粒子位置和速度
def initialize_particles(num_particles, disc_radius, mass, temperature):
    positions = np.random.uniform(-disc_radius, disc_radius, (num_particles, 2))
    velocities = np.random.normal(0, np.sqrt(k_B * temperature / mass), (num_particles, 2))
    distances = np.sqrt(np.sum(positions**2, axis=1))
    inside_disc = distances <= disc_radius
    return positions[inside_disc], velocities[inside_disc]

# 电子和离子初始化
electron_positions, electron_velocities = initialize_particles(num_particles, disc_radius, mass_electron, temperature)
ion_positions, ion_velocities = initialize_particles(num_particles, disc_radius, mass_ion, temperature)
num_particles_e = len(electron_positions)
num_particles_i = len(ion_positions)
charge_electron_array = np.full(num_particles_e, charge_electron)
charge_ion_array = np.full(num_particles_i, charge_ion)
mass_electron_array = np.full(num_particles_e, mass_electron)
mass_ion_array = np.full(num_particles_i, mass_ion)

# 电势和电场初始化
phi = np.zeros((grid_size, grid_size))
electric_field = np.zeros((grid_size, grid_size, 2))

# 泊松方程求解
@njit
def solve_poisson(phi, rho):
    for _ in range(100):  # 迭代次数
        phi_new = phi.copy()
        phi_new[1:-1, 1:-1] = 0.25 * (phi[:-2, 1:-1] + phi[2:, 1:-1] + phi[1:-1, :-2] + phi[1:-1, 2:] - rho[1:-1, 1:-1] * cell_size**2)
        phi = phi_new
    return phi

# 计算电场
@njit
def compute_electric_field(phi):
    ex = (phi[:-2, 1:-1] - phi[2:, 1:-1]) / (2 * cell_size)
    ey = (phi[1:-1, :-2] - phi[1:-1, 2:]) / (2 * cell_size)
    return ex, ey

# 计算库仑加速度（只考虑德拜半径内的粒子）
@njit(parallel=True)
def compute_coulomb_acceleration(positions, charges, masses, debye_radius_grid):
    num_particles = len(positions)
    acceleration = np.zeros_like(positions)
    
    for i in prange(num_particles):
        dx = positions - positions[i]
        distance = np.sqrt(np.sum(dx**2, axis=1))
        within_debye_radius = distance <= debye_radius_grid * cell_size
        
        for j in range(num_particles):
            if j == i or not within_debye_radius[j]:
                continue
            r = distance[j]
            force = charges[i] * charges[j] / (4 * np.pi * epsilon_0 * r**3)
            acceleration[i] += force * dx[j] / masses[i]
        
    return acceleration

# 计算电荷密度
def compute_charge_density(positions, charge, grid_size, domain_size, cell_size):
    rho = np.zeros((grid_size, grid_size))
    bin_edges = [np.linspace(-domain_size / 2, domain_size / 2, grid_size + 1)] * 2
    hist, _, _ = np.histogram2d(positions[:, 0], positions[:, 1], bins=bin_edges, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
    rho = hist * charge
    return rho

# 初始化电荷密度
rho_e = compute_charge_density(electron_positions, charge_electron, grid_size, domain_size, cell_size)
rho_i = compute_charge_density(ion_positions, charge_ion, grid_size, domain_size, cell_size)
rho = rho_e + rho_i

# 求解初始电势
phi = solve_poisson(phi, rho)

# 计算初始电场
ex, ey = compute_electric_field(phi)
electric_field[1:-1, 1:-1, 0] = ex
electric_field[1:-1, 1:-1, 1] = ey

# 模拟
for step in range(num_steps + 1):  # 包括初始状态
    if step > 0:
        # 计算电子和离子之间的库仑加速度
        electron_acceleration = compute_coulomb_acceleration(electron_positions, charge_electron_array, mass_electron_array, debye_radius_grid)
        ion_acceleration = compute_coulomb_acceleration(ion_positions, charge_ion_array, mass_ion_array, debye_radius_grid)

        # 更新电子位置和速度
        temp_positions_e = electron_positions.copy()
        electron_positions += electron_velocities * dt + 0.5 * electron_acceleration * dt**2
        electron_velocities += electron_acceleration * dt
        
        # 确保电子不会离开圆盘边界
        distances_e = np.sqrt(np.sum(electron_positions**2, axis=1))
        outside_disc_e = distances_e > disc_radius
        electron_positions[outside_disc_e] = temp_positions_e[outside_disc_e]
        electron_velocities[outside_disc_e] *= -1  # 反弹

        # 更新离子位置和速度
        temp_positions_i = ion_positions.copy()
        ion_positions += ion_velocities * dt + 0.5 * ion_acceleration * dt**2
        ion_velocities += ion_acceleration * dt
        
        # 确保离子不会离开圆盘边界
        distances_i = np.sqrt(np.sum(ion_positions**2, axis=1))
        outside_disc_i = distances_i > disc_radius
        ion_positions[outside_disc_i] = temp_positions_i[outside_disc_i]
        ion_velocities[outside_disc_i] *= -1  # 反弹

        # 计算新的电荷密度
        rho_e = compute_charge_density(electron_positions, charge_electron, grid_size, domain_size, cell_size)
        rho_i = compute_charge_density(ion_positions, charge_ion, grid_size, domain_size, cell_size)
        rho = rho_e + rho_i

        # 解决泊松方程
        phi = solve_poisson(phi, rho)

        # 计算电场
        ex, ey = compute_electric_field(phi)
        electric_field[1:-1, 1:-1, 0] = ex
        electric_field[1:-1, 1:-1, 1] = ey

    # 可视化
    if step % 1 == 0:
        fig, axs = plt.subplots(1, 3, figsize=(18, 6))
        axs[0].cla()
        axs[1].cla()
        axs[2].cla()
        
        hist_e, xedges, yedges = np.histogram2d(electron_positions[:, 0], electron_positions[:, 1], bins=grid_size, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
        hist_i, _, _ = np.histogram2d(ion_positions[:, 0], ion_positions[:, 1], bins=grid_size, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
        
        im1 = axs[0].imshow(hist_i.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='Reds', vmin=0, vmax=np.max(hist_i))
        axs[0].set_title('Ion Distribution')

        im2 = axs[1].imshow(hist_e.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='Blues', vmin=0, vmax=np.max(hist_e))
        axs[1].set_title('Electron Distribution')

        magnitude_e_field = np.sqrt(electric_field[:, :, 0]**2 + electric_field[:, :, 1]**2)
        im3 = axs[2].imshow(magnitude_e_field.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='plasma', vmin=0, vmax=np.max(magnitude_e_field))
        axs[2].set_title('Electric Field Magnitude')

        fig.colorbar(im1, ax=axs[0], orientation='vertical', label='Number of Ions')
        fig.colorbar(im2, ax=axs[1], orientation='vertical', label='Number of Electrons')
        fig.colorbar(im3, ax=axs[2], orientation='vertical', label='Electric Field Magnitude')

        plt.tight_layout()
        plt.savefig(f'step_{step:04d}.png')  # 保存图像
        plt.clf()  # 清理当前图像
        plt.close(fig)
print('所有图像已保存')


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange

# 参数设置
num_particles = 1000000
domain_size = 10.0  # cm
disc_radius = 3.0  # cm
temperature = 1000  # K
mass_electron = 9.10938356e-31  # kg
mass_ion = 1.6726219e-27  # kg (质子质量)
charge_electron = -1.60217662e-19  # C
charge_ion = 1.60217662e-19  # C
k_B = 1.38064852e-23  # J/K
num_steps = 1000
dt = 8.3e-10  # s (8.3e-7 / 1000)
epsilon_0 = 8.854187817e-12  # F/m (真空介电常数)

# 初始化粒子位置和速度
def initialize_particles(num_particles, disc_radius, mass, temperature):
    positions = np.random.uniform(-disc_radius, disc_radius, (num_particles, 2))
    velocities = np.random.normal(0, np.sqrt(k_B * temperature / mass), (num_particles, 2))
    distances = np.sqrt(np.sum(positions**2, axis=1))
    inside_disc = distances <= disc_radius
    return positions[inside_disc], velocities[inside_disc]

# 电子和离子初始化
electron_positions, electron_velocities = initialize_particles(num_particles, disc_radius, mass_electron, temperature)
ion_positions, ion_velocities = initialize_particles(num_particles, disc_radius, mass_ion, temperature)
num_particles_e = len(electron_positions)
num_particles_i = len(ion_positions)
charge_electron_array = np.full(num_particles_e, charge_electron)
charge_ion_array = np.full(num_particles_i, charge_ion)
mass_electron_array = np.full(num_particles_e, mass_electron)
mass_ion_array = np.full(num_particles_i, mass_ion)

# 电势和电场初始化
grid_size = 60
cell_size = domain_size / grid_size  # cm
phi = np.zeros((grid_size, grid_size))
electric_field = np.zeros((grid_size, grid_size, 2))

# 泊松方程求解
@njit
def solve_poisson(phi, rho, cell_size):
    for _ in range(100):  # 迭代次数
        phi_new = phi.copy()
        phi_new[1:-1, 1:-1] = 0.25 * (phi[:-2, 1:-1] + phi[2:, 1:-1] + phi[1:-1, :-2] + phi[1:-1, 2:] - rho[1:-1, 1:-1] * cell_size**2)
        phi = phi_new
    return phi

# 计算电场
@njit
def compute_electric_field(phi, cell_size):
    ex = (phi[:-2, 1:-1] - phi[2:, 1:-1]) / (2 * cell_size)
    ey = (phi[1:-1, :-2] - phi[1:-1, 2:]) / (2 * cell_size)
    return ex, ey

# 计算电荷密度
@njit
def compute_charge_density(positions, charge, grid_size, domain_size, cell_size):
    rho = np.zeros((grid_size, grid_size))
    for pos in positions:
        x_idx = int((pos[0] + domain_size / 2) // cell_size)
        y_idx = int((pos[1] + domain_size / 2) // cell_size)
        if 0 <= x_idx < grid_size and 0 <= y_idx < grid_size:
            rho[x_idx, y_idx] += charge
    return rho

# 插值电场到粒子位置
@njit(parallel=True)
def interpolate_electric_field(positions, electric_field, grid_size, domain_size, cell_size):
    e_field_particles = np.zeros_like(positions)
    for i in prange(len(positions)):
        x_idx = int((positions[i, 0] + domain_size / 2) // cell_size)
        y_idx = int((positions[i, 1] + domain_size / 2) // cell_size)
        if 0 <= x_idx < grid_size and 0 <= y_idx < grid_size:
            e_field_particles[i] = electric_field[x_idx, y_idx]
    return e_field_particles

# 初始化电荷密度
rho_e = compute_charge_density(electron_positions, charge_electron, grid_size, domain_size, cell_size)
rho_i = compute_charge_density(ion_positions, charge_ion, grid_size, domain_size, cell_size)
rho = rho_e + rho_i

# 求解初始电势
phi = solve_poisson(phi, rho, cell_size)

# 计算初始电场
ex, ey = compute_electric_field(phi, cell_size)
electric_field[1:-1, 1:-1, 0] = ex
electric_field[1:-1, 1:-1, 1] = ey

# 模拟
for step in range(num_steps + 1):  # 包括初始状态
    if step > 0:
        # 插值电场到粒子位置
        e_field_electron = interpolate_electric_field(electron_positions, electric_field, grid_size, domain_size, cell_size)
        e_field_ion = interpolate_electric_field(ion_positions, electric_field, grid_size, domain_size, cell_size)

        # 计算电子和离子之间的电场加速度
        electron_acceleration = charge_electron * e_field_electron / mass_electron
        ion_acceleration = charge_ion * e_field_ion / mass_ion

        # 更新电子位置和速度
        temp_positions_e = electron_positions.copy()
        electron_positions += electron_velocities * dt + 0.5 * electron_acceleration * dt**2
        electron_velocities += electron_acceleration * dt
        
        # 确保电子不会离开圆盘边界
        distances_e = np.sqrt(np.sum(electron_positions**2, axis=1))
        outside_disc_e = distances_e > disc_radius
        electron_positions[outside_disc_e] = temp_positions_e[outside_disc_e]
        electron_velocities[outside_disc_e] *= -1  # 反弹

        # 更新离子位置和速度
        temp_positions_i = ion_positions.copy()
        ion_positions += ion_velocities * dt + 0.5 * ion_acceleration * dt**2
        ion_velocities += ion_acceleration * dt
        
        # 确保离子不会离开圆盘边界
        distances_i = np.sqrt(np.sum(ion_positions**2, axis=1))
        outside_disc_i = distances_i > disc_radius
        ion_positions[outside_disc_i] = temp_positions_i[outside_disc_i]
        ion_velocities[outside_disc_i] *= -1  # 反弹

        # 计算新的电荷密度
        rho_e = compute_charge_density(electron_positions, charge_electron, grid_size, domain_size, cell_size)
        rho_i = compute_charge_density(ion_positions, charge_ion, grid_size, domain_size, cell_size)
        rho = rho_e + rho_i

        # 解决泊松方程
        phi = solve_poisson(phi, rho, cell_size)

        # 计算电场
        ex, ey = compute_electric_field(phi, cell_size)
        electric_field[1:-1, 1:-1, 0] = ex
        electric_field[1:-1, 1:-1, 1] = ey

    # 可视化
    if step % 10 == 0:  # 可视化间隔
        fig, axs = plt.subplots(1, 3, figsize=(18, 6))
        axs[0].cla()
        axs[1].cla()
        axs[2].cla()
        
        hist_e, xedges, yedges = np.histogram2d(electron_positions[:, 0], electron_positions[:, 1], bins=grid_size, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
        hist_i, _, _ = np.histogram2d(ion_positions[:, 0], ion_positions[:, 1], bins=grid_size, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
        
        im1 = axs[0].imshow(hist_i.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='Reds', vmin=0, vmax=np.max(hist_i))
        axs[0].set_title('Ion Distribution')

        im2 = axs[1].imshow(hist_e.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='Blues', vmin=0, vmax=np.max(hist_e))
        axs[1].set_title('Electron Distribution')

        magnitude_e_field = np.sqrt(electric_field[:, :, 0]**2 + electric_field[:, :, 1]**2)
        im3 = axs[2].imshow(magnitude_e_field.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='plasma', vmin=0, vmax=np.max(magnitude_e_field))
        axs[2].set_title('Electric Field Magnitude')

        fig.colorbar(im1, ax=axs[0], orientation='vertical', label='Number of Ions')
        fig.colorbar(im2, ax=axs[1], orientation='vertical', label='Number of Electrons')
        fig.colorbar(im3, ax=axs[2], orientation='vertical', label='Electric Field Magnitude')

        plt.tight_layout()
        plt.savefig(f'step_{step:04d}.png')  # 保存图像
        plt.clf()  # 清理当前图像
        plt.close(fig)

print('所有图像已保存')


KeyboardInterrupt: 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange

# 参数设置
num_particles = 100000
domain_size = 10.0  # cm
disc_radius = 3.0  # cm
temperature = 1000  # K
mass_electron = 9.10938356e-31  # kg
mass_ion = 1.6726219e-27  # kg (质子质量)
charge_electron = -1.60217662e-19  # C
charge_ion = 1.60217662e-19  # C
k_B = 1.38064852e-23  # J/K
num_steps = 1000
dt = 8.3e-10  # s (8.3e-7 / 1000)
epsilon_0 = 8.854187817e-12  # F/m (真空介电常数)

# 计算粒子密度（m^-3）
volume = (2 * disc_radius / 100)**2  # 转换为m²
n = num_particles / volume  # 粒子密度 #只考虑初始情况等离子体密度

# 计算德拜长度
debye_length = np.sqrt(epsilon_0 * k_B * temperature / (n * charge_electron**2))

# 转换德拜长度到网格单元
grid_size = 60
cell_size = domain_size / grid_size  # cm
debye_radius_grid = int(np.ceil(debye_length / (cell_size / 100)))  # 转换为cm，再转换为网格单元


# 初始化粒子位置和速度
def initialize_particles(num_particles, disc_radius, mass, temperature):
    positions = np.random.uniform(-disc_radius, disc_radius, (num_particles, 2))
    velocities = np.random.normal(0, np.sqrt(k_B * temperature / mass), (num_particles, 2))
    distances = np.sqrt(np.sum(positions**2, axis=1))
    inside_disc = distances <= disc_radius
    return positions[inside_disc], velocities[inside_disc]

# 电子和离子初始化
electron_positions, electron_velocities = initialize_particles(num_particles, disc_radius, mass_electron, temperature)
ion_positions, ion_velocities = initialize_particles(num_particles, disc_radius, mass_ion, temperature)
num_particles_e = len(electron_positions)
num_particles_i = len(ion_positions)
charge_electron_array = np.full(num_particles_e, charge_electron)
charge_ion_array = np.full(num_particles_i, charge_ion)
mass_electron_array = np.full(num_particles_e, mass_electron)
mass_ion_array = np.full(num_particles_i, mass_ion)

# 电势和电场初始化
phi = np.zeros((grid_size, grid_size))
electric_field = np.zeros((grid_size, grid_size, 2))

# 泊松方程求解
@njit
def solve_poisson(phi, rho, num_iterations=100):
    for _ in range(num_iterations):  # 迭代次数
        phi_new = phi.copy()
        phi_new[1:-1, 1:-1] = 0.25 * (phi[:-2, 1:-1] + phi[2:, 1:-1] + phi[1:-1, :-2] + phi[1:-1, 2:] - rho[1:-1, 1:-1] * cell_size**2)
        phi = phi_new
    return phi

# 计算电场
@njit
def compute_electric_field(phi, cell_size):
    ex = (phi[:-2, 1:-1] - phi[2:, 1:-1]) / (2 * cell_size)
    ey = (phi[1:-1, :-2] - phi[1:-1, 2:]) / (2 * cell_size)
    return ex, ey

# 计算库仑加速度（只考虑德拜半径内的粒子）
@njit(parallel=True)
def compute_coulomb_acceleration(positions, charges, masses, debye_radius_grid, cell_size):
    num_particles = len(positions)
    acceleration = np.zeros_like(positions)
    
    for i in prange(num_particles):
        dx = positions - positions[i]
        distance = np.sqrt(np.sum(dx**2, axis=1))
        within_debye_radius = distance <= debye_radius_grid * cell_size
        
        for j in range(num_particles):
            if j == i or not within_debye_radius[j]:
                continue
            r = distance[j]
            if r == 0:  # 避免除以零
                continue
            force = charges[i] * charges[j] / (4 * np.pi * epsilon_0 * r**3)
            acceleration[i] += force * dx[j] / masses[i]
        
    return acceleration

# 计算电荷密度
def compute_charge_density(positions, charge, grid_size, domain_size, cell_size):
    rho = np.zeros((grid_size, grid_size))
    bin_edges = [np.linspace(-domain_size / 2, domain_size / 2, grid_size + 1)] * 2
    hist, _, _ = np.histogram2d(positions[:, 0], positions[:, 1], bins=bin_edges, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
    rho = hist * charge
    return rho

# 初始化电荷密度
rho_e = compute_charge_density(electron_positions, charge_electron, grid_size, domain_size, cell_size)
rho_i = compute_charge_density(ion_positions, charge_ion, grid_size, domain_size, cell_size)
rho = rho_e + rho_i

# 求解初始电势
phi = solve_poisson(phi, rho)

# 计算初始电场
ex, ey = compute_electric_field(phi, cell_size)
electric_field[1:-1, 1:-1, 0] = ex
electric_field[1:-1, 1:-1, 1] = ey

# 更新粒子位置和速度
def update_positions_velocities(positions, velocities, accelerations, dt, radius):
    new_positions = positions + velocities * dt + 0.5 * accelerations * dt**2
    new_velocities = velocities + accelerations * dt

    # 确保粒子不会离开圆盘边界
    distances = np.sqrt(np.sum(new_positions**2, axis=1))
    outside_disc = distances > radius

    new_positions[outside_disc] = positions[outside_disc]  # 保持位置不变
    new_velocities[outside_disc] *= -1  # 反弹速度

    return new_positions, new_velocities

# 模拟
for step in range(num_steps + 1):  # 包括初始状态
    if step > 0:
        # 计算电子和离子之间的库仑加速度
        electron_acceleration = compute_coulomb_acceleration(electron_positions, charge_electron_array, mass_electron_array, debye_radius_grid, cell_size)
        ion_acceleration = compute_coulomb_acceleration(ion_positions, charge_ion_array, mass_ion_array, debye_radius_grid, cell_size)

        # 更新电子位置和速度
        electron_positions, electron_velocities = update_positions_velocities(electron_positions, electron_velocities, electron_acceleration, dt, disc_radius)

        # 更新离子位置和速度
        ion_positions, ion_velocities = update_positions_velocities(ion_positions, ion_velocities, ion_acceleration, dt, disc_radius)

        # 计算新的电荷密度
        rho_e = compute_charge_density(electron_positions, charge_electron, grid_size, domain_size, cell_size)
        rho_i = compute_charge_density(ion_positions, charge_ion, grid_size, domain_size, cell_size)
        rho = rho_e + rho_i

        # 解决泊松方程
        phi = solve_poisson(phi, rho)

        # 计算电场
        ex, ey = compute_electric_field(phi, cell_size)
        electric_field[1:-1, 1:-1, 0] = ex
        electric_field[1:-1, 1:-1, 1] = ey

    # 可视化
    if step % 1 == 0:
        fig, axs = plt.subplots(1, 3, figsize=(18, 6))
        axs[0].cla()
        axs[1].cla()
        axs[2].cla()
        
        hist_e, xedges, yedges = np.histogram2d(electron_positions[:, 0], electron_positions[:, 1], bins=grid_size, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
        hist_i, _, _ = np.histogram2d(ion_positions[:, 0], ion_positions[:, 1], bins=grid_size, range=[[-domain_size / 2, domain_size / 2], [-domain_size / 2, domain_size / 2]])
        
        im1 = axs[0].imshow(hist_i.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='Reds', vmin=0, vmax=np.max(hist_i))
        axs[0].set_title('Ion Distribution')

        im2 = axs[1].imshow(hist_e.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='Blues', vmin=0, vmax=np.max(hist_e))
        axs[1].set_title('Electron Distribution')

        magnitude_e_field = np.sqrt(electric_field[:, :, 0]**2 + electric_field[:, :, 1]**2)
        im3 = axs[2].imshow(magnitude_e_field.T, origin='lower', extent=[-domain_size / 2, domain_size / 2, -domain_size / 2, domain_size / 2], cmap='plasma', vmin=0, vmax=np.max(magnitude_e_field))
        axs[2].set_title('Electric Field Magnitude')

        fig.colorbar(im1, ax=axs[0], orientation='vertical', label='Number of Ions')
        fig.colorbar(im2, ax=axs[1], orientation='vertical', label='Number of Electrons')
        fig.colorbar(im3, ax=axs[2], orientation='vertical', label='Electric Field Magnitude')

        plt.tight_layout()
        plt.savefig(f'step_{step:04d}.png')  # 保存图像
        plt.clf()  # 清理当前图像
        plt.close(fig)

print('所有图像已保存')
